## Tensor Nets

In this notebook we provide an example of how to build a simple Tensor Net (see https://arxiv.org/abs/1509.06569).

The main ingredient is the so-called TT-Matrix, a generalization of the Kronecker product matrices, i.e. matrices of the form 
$$A = A_1 \otimes A_2 \cdots \otimes A_n$$

In `t3f` TT-Matrices are represented using the `TensorTrain` class.

In [1]:
import t3f
import tensorflow as tf

In [2]:
W = t3f.random_matrix([[4, 7, 4, 7], [5, 5, 5, 5]], tt_rank=2)

print(W)

A TT-Matrix of size 784 x 625, underlying tensor shape: (4, 7, 4, 7) x (5, 5, 5, 5), TT-ranks: (1, 2, 2, 2, 1)


Using TT-Matrices we can compactly represent densely connected layers in neural networks, which allows us to greatly reduce number of parameters. Matrix multiplication can be handled by the `t3f.matmul` method which allows for multiplying dense (ordinary) matrices and TT-Matrices. Very simple neural network could look as following:

In [3]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int64, [None])

W1 = t3f.get_variable('W1', initializer=W) 
b1 = tf.get_variable('b1', shape=[625])
h1 = t3f.matmul(x, W1) + b1

W2 = tf.get_variable('W2', shape=[625, 10])
b2 = tf.get_variable('b2', shape=[10])
h2 = tf.matmul(h1, W2) + b2

y_ = tf.one_hot(y, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=h2))

For convenience we have implemented a layer analogous to *Keras* `Dense` layer but with a TT-Matrix instead of an ordinary matrix. An example of fully trainable net is provided below.

In [4]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import t3f
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from utils import TTDense

Using TensorFlow backend.


In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Some preprocessing...

In [21]:
x_train = x_train.reshape((-1, 784)) / 255.0
x_test = x_test.reshape((-1, 784)) / 255.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [34]:
model = Sequential()

In [35]:
model.add(TTDense(row_dims=[7, 4, 7, 4], column_dims=[5, 5, 5, 5], input_shape=(784, ), tt_rank=4, activation='relu'))

In [38]:
model.add(Dropout(0.5))
model.add(TTDense(row_dims=[5, 5, 5, 5], column_dims=[5, 5, 5, 5], tt_rank=4, activation='relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tt_dense_5 (TTDense)         (None, 625)               1725      
_________________________________________________________________
dropout_1 (Dropout)          (None, 625)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 625)               0         
_________________________________________________________________
tt_dense_7 (TTDense)         (None, 625)               1625      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                6260      
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
Total params: 9,610
Trainable params: 9,610
Non-trainable params: 0
_________________________________________________________________


Note that we only have $1725$ parameters instead of $784 * 625 = 490000$.

In [40]:
optimizer = optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
36672/60000 [=================>............] - ETA: 17s - loss: 1.0203 - acc: 0.6607

KeyboardInterrupt: 